In [5]:
import pygame as pg
import pygame.gfxdraw
import math
import random as rand


pg.init()
# pygame.key.set_repeat(30)
pg.display.set_caption("Raycaster")
resolution = 4
sw = 840 // resolution
sh = 480 // resolution
screen = pg.display.set_mode((sw * resolution, sh * resolution))
clock = pg.time.Clock()

map = [
    1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
    1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,
    1,0,0,0,0,0,0,0,1,1,1,0,0,0,1,
    1,0,0,0,0,0,0,0,1,1,1,0,0,0,1,
    1,0,0,0,0,0,0,0,1,1,1,0,0,0,1,
    1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,
    1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,
    1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,
    1,0,0,0,0,0,0,2,0,0,0,0,0,0,1,
    1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,
    1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,
    1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,
    1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
]

texture = pg.image.load("../assets/background.jpg")
textpixels = pygame.PixelArray(texture)

sprite = pg.image.load("../assets/sprite.jpg")
spritePixel = pygame.PixelArray(sprite)

mw = 15
mh = 13

ppos = [map.index(2) % mw, map.index(2) // mw]
pdir = 0
ppitch = 0
proll = 0
pfov = 3.14159 / 4.0

depth = 16.0

wallScale = 1

objs = [{"pos": (4, 3)}]

isRunning = True
while isRunning:

    rand.seed(f"{ppos[0]},{ppos[1]},{pdir}")
    for x in range(sw):
        rayAngle = (pdir - pfov / 2) + (x / sw) * pfov
        dist = 0
        hit = False
        eyeX = math.sin(rayAngle)
        exeY = math.cos(rayAngle)
        dtw = 0
        sampleX = 0

        while not hit and dtw < depth:
            dtw += 0.1
            testX = int(ppos[0] + eyeX * dtw)
            testY = int(ppos[1] + exeY * dtw)
            if testX < 0 or testX >= mw or testY < 0 or testY >= mh:
                hit = True
                dtw = depth
            else:
                if map[testY * mw + testX] == 1:
                    hit = True 

                    bmidX = testX + 0.5
                    bmidY = testY + 0.5
                    tpX = ppos[0] + eyeX * dtw
                    tpY = ppos[1] + exeY * dtw
                    tangle = math.atan2((tpY - bmidY), (tpX - bmidX))

                    if tangle >= -math.pi * 0.25 and tangle < math.pi * 0.25:
                        sampleX = tpY - testY
                    elif tangle >= math.pi * 0.25 and tangle < math.pi * 0.75:
                        sampleX = tpX - testX
                    elif tangle < -math.pi * 0.25 and tangle >= -math.pi * 0.75:
                        sampleX = tpX - testX
                    elif tangle >= math.pi * 0.75 or tangle < -math.pi * 0.75:
                        sampleX = tpY - testY

        ceil = sh / 2 - sh / (dtw / wallScale)
        ceil *= 0.8
        floor = sh - ceil

        ceil = ceil + math.sin(ppitch) * 10 * dtw
        floor = floor + math.sin(ppitch) * 10 * dtw

        ceil = ceil + math.sin(proll) * (sw/2-x)
        floor = floor + math.sin(proll) * (sw/2-x)

        for y in range(sh):
            if y < ceil:
                col = (0,0,0)
                if rand.randint(0, 100) == 10:
                    col = (200, 200, 200)
                pg.gfxdraw.box(screen, (x*resolution, y*resolution, resolution, resolution), col) 
            elif y > ceil and y <= floor:
                sampleY = (y - ceil) / (floor - ceil)
                color = pg.Color(textpixels[int(sampleX * texture.get_width())-1][int(sampleY * texture.get_height())-1])
                # pg.gfxdraw.pixel(screen, x*4, y*4, (color.b, color.b, color.b)) 
                pg.gfxdraw.box(screen, (x*resolution, y*resolution, resolution , resolution), (color.b, color.b, color.b))
            else:
                fy = (y-floor) / ceil
                # wx = ppos[0] * math.sin(pdir) + fy * dtw
                # wy = ppos[1] * math.cos(pdir) + fy * dtw
                # color = pg.Color(textpixels[int(wx/mw * texture.get_width())-1][int(wy/mh * texture.get_height())-1])
                pg.gfxdraw.box(screen, (x*resolution, y*resolution, resolution, resolution), (128*fy,128*fy,128*fy)) 

        for o in objs:
            opos = o["pos"]

    pg.display.update()
    clock.tick(30)

    for e in pg.event.get():
        if e.type == pg.QUIT:
            pg.quit()
            isRunning = False

        if e.type == pygame.KEYDOWN:
            if e.key == pygame.K_q:
                pdir -= 0.1
            if e.key == pygame.K_e:
                pdir += 0.1

            if e.key == pygame.K_w:
                ppos[0] += math.sin(pdir) * 0.1
                ppos[1] += math.cos(pdir) * 0.1
            if e.key == pygame.K_s:
                ppos[0] -= math.sin(pdir) * 0.1
                ppos[1] -= math.cos(pdir) * 0.1

            if e.key == pygame.K_i:
                ppitch += 0.1
            if e.key == pygame.K_k:
                ppitch -= 0.1
            if e.key == pygame.K_j:
                proll += 0.1
            if e.key == pygame.K_l:
                proll -= 0.1

            